In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datasetsforecast.m4 import M4
from config_models import MODEL_LIST

from neuralforecast.losses.numpy import mae, mse, smape, mqloss

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpx0i3xtiv
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpx0i3xtiv/_remote_module_non_scriptable.py


In [2]:
Y_df, a, b = M4.load(directory='./', group='Monthly', cache=True)
y = Y_df.groupby('unique_id').tail(12).reset_index(drop=True)['y']

## Auto

M4

In [3]:
# source_dataset = 'M4'
# target_dataset = 'M4'
# experiment_id = '20230606'

# models = ['automlp', 'autonhits','autotft','autopatchtst', 'autotcn', 'autolstm']
# model_parse = {
#                 'autonhits': 'NHITS',
#                 'autotft': 'TFT',
#                 'autopatchtst': 'PatchTST',
#                 'automlp': 'MLP',
#                 'autotcn': 'TCN',
#                 'autolstm': 'LSTM',
#               }

# for model in models:
#     model_name = model_parse[model]
#     print('Model:', model_name)
#     results_df = pd.read_csv(f'./results/forecasts/{target_dataset}/{model}_{source_dataset}_{experiment_id}.csv')
#     if target_dataset == 'M4':
#       results_df['y'] = y
#     #results_df = results_df.groupby('unique_id').tail(12)

#     smape_loss = smape(results_df['y'], results_df.iloc[:,3])
#     mae_loss = mae(results_df['y'], results_df.iloc[:,3])
#     print('smape:', smape_loss)
#     print('mae:', mae_loss)

In [4]:
# source_dataset = 'M4'
# target_dataset = 'M3'
# experiment_id = '20230606'

# models = ['automlp', 'autonhits','autotft','autopatchtst', 'autotcn', 'autolstm']
# model_parse = {
#                 'autonhits': 'NHITS',
#                 'autotft': 'TFT',
#                 'autopatchtst': 'PatchTST',
#                 'automlp': 'MLP',
#                 'autotcn': 'TCN',
#                 'autolstm': 'LSTM',
#               }

# for model in models:
#     model_name = model_parse[model]
#     print('Model:', model_name)
#     results_df = pd.read_csv(f'./results/forecasts/{target_dataset}/{model}_{source_dataset}_{experiment_id}.csv')
#     if target_dataset == 'M4':
#       results_df['y'] = y
#     #results_df = results_df.groupby('unique_id').tail(12)

#     smape_loss = smape(results_df['y'], results_df[model_name])
#     mae_loss = mae(results_df['y'], results_df[model_name])
#     print('smape:', smape_loss)
#     print('mae:', mae_loss)

# Fixed

In [5]:
source_dataset = 'M4'
target_dataset = 'M3'
experiment_id = '20230621'

for model in MODEL_LIST:
    print('Model:', model)
    results_df = pd.read_csv(f'./results/forecasts/{target_dataset}/{model}_{source_dataset}_{experiment_id}.csv').reset_index(drop=True)
    #print('results_df', results_df.columns)
    if target_dataset == 'M4':
      results_df['y'] = y
    results_df = results_df.groupby('unique_id').tail(12)

    smape_loss = smape(results_df['y'], results_df.iloc[:,8])
    mae_loss = mae(results_df['y'], results_df.iloc[:,8])
    mq_loss = mqloss(y=results_df['y'], y_hat=results_df.iloc[:,4:13], quantiles=np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]))
    print('smape:', smape_loss)
    print('mae:', mae_loss)
    print('mqloss:', mq_loss)

Model: nhits_15_512
smape: 0.14747429369933274
mae: 656.23565581285
mqloss: 265.41630900681207
Model: nhits_30_1024
smape: 0.14541996181886246
mae: 649.5826373482727
mqloss: 264.18316240930994
Model: nhits_30_2048
smape: 0.14645092800328163
mae: 650.5023547718254
mqloss: 263.1996163726084
Model: patchtst_128_3
smape: 0.1520823539944732
mae: 704.2193241100606
mqloss: 278.0112706847326
Model: patchtst_512_6
smape: 0.1474111581489757
mae: 668.2095220694445
mqloss: 270.0446589497616
Model: patchtst_1024_6
smape: 0.14876738406166748
mae: 681.4438905987396
mqloss: 277.96903409246147
Model: tft_128
smape: 0.1447438766587527
mae: 652.6071338106909
mqloss: 265.77370940031113
Model: tft_512
smape: 0.14560140590324705
mae: 650.7781857201213
mqloss: 264.5692171961994
Model: tft_1024
smape: 0.1466067503614752
mae: 656.591426131536
mqloss: 267.31999285811315
Model: mlp_512_8
smape: 0.14631907943211472
mae: 657.2981284926472
mqloss: 265.3993284891515
Model: mlp_2048_32
smape: 0.17495658698632544
mae:

In [ ]:
results_df

In [ ]:
# for i in range(1):
#     unique_id = results_df['unique_id'].unique()[np.random.randint(1)]
#     plot_df = results_df.query('unique_id == @unique_id')

#     plt.figure(figsize=(20, 5))
#     plt.plot(plot_df['y'], label='y')
#     plt.plot(plot_df[model_name], label='y_hat')
#     plt.show()

In [ ]:
import pandas as pd
import numpy as npx

from datasetsforecast.m4 import M4
from datasetsforecast.m3 import M3
from datasetsforecast.long_horizon import LongHorizon
from datasetsforecast.losses import mae
from statsforecast.utils import AirPassengersDF

In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA, AutoETS, Naive

In [ ]:
target_dataset = 'TrafficL'
horizon = 18

In [ ]:
if (target_dataset == 'AirPassengers'):
    Y_df_target = AirPassengersDF.copy()
    Y_df_target['ds'] = pd.to_datetime(Y_df_target['ds'])
    test_size = horizon
    frequency = 'M'
elif (target_dataset == 'M3'):
    Y_df_target, *_ = M3.load(directory='./', group='Monthly')
    Y_df_target['ds'] = pd.to_datetime(Y_df_target['ds'])
    frequency = 'M'
    test_size = horizon
elif (target_dataset == 'M4'):
    Y_df_target, *_ = M4.load(directory='./', group='Monthly', cache=True)
    Y_df_target['ds'] = pd.to_datetime(Y_df_target['ds'])
    frequency = 'M'
    test_size = horizon
elif (target_dataset == 'ILI'):
    Y_df_target, _, _ = LongHorizon.load(directory='./', group='ILI')
    Y_df_target['ds'] = np.repeat(np.array(range(len(Y_df_target)//7)), 7)
    test_size = horizon
    frequency = 'W'
elif (target_dataset == 'TrafficL'):
    Y_df_target, _, _ = LongHorizon.load(directory='./', group='TrafficL')
    Y_df_target['ds'] = np.repeat(np.array(range(len(Y_df_target)//862)), 862)
    test_size = horizon

In [ ]:
unique_ids = Y_df_target['unique_id'].unique()
unique_ids = np.random.choice(unique_ids,size=20)
Y_df_target = Y_df_target[Y_df_target['unique_id'].isin(unique_ids)].reset_index(drop=True)
Y_df_target = Y_df_target.groupby('unique_id').tail(200).reset_index(drop=True)

In [ ]:
Y_df_target

In [ ]:
season_length = 12 # Monthly data 
#season_length = 24 # Monthly data 

# Include the models you imported
models = [
    AutoARIMA(season_length=season_length),
    #AutoETS(season_length=season_length),
    Naive()
]

# Instansiate the StatsForecast class as sf
sf = StatsForecast(
    df=Y_df_target,
    models=models,
    freq='M', 
    n_jobs=-1
)

crossvalidation_df = sf.cross_validation(
    df = Y_df_target,
    h = horizon,
    step_size = 1,
    n_windows = 1
  )

In [ ]:
crossvalidation_df = crossvalidation_df.reset_index(drop=True)
crossvalidation_df

In [ ]:
for model_name in ['AutoARIMA', 'Naive']:
    print('Model:', model_name)
    mae_loss = mae(crossvalidation_df['y'], crossvalidation_df[model_name])
    print('mae:', mae_loss)